In [1]:
import numpy as np
import pandas as pd
from reinforced_concrete.sections import create_concrete_material, create_steel_material, Bars, ReinforcedConcreteSection, InternalForces

from reinforced_concrete.sls import sls
from reinforced_concrete.ULS import computeVero

In [2]:
EXPORT_FOLDER = "export/"
# TODO mettere il os.mkdir()

# Single section:

In [3]:
cls  = create_concrete_material("EC2","C25/30") 
steel  = create_steel_material("NTC18","B450C")

section_1 = ReinforcedConcreteSection(
    b=600, 
    d=355, 
    d1=45, 
    d2=45, 
    concrete_material=cls, 
    As=Bars(n_bars=12, diameter=18, steel_material=steel), 
    As1=Bars(n_bars=9, diameter=18, steel_material=steel), 
    internal_forces=InternalForces(M=310*10**6, N=-150*10**3), 
    name="sec1"
    )

section_1 = ReinforcedConcreteSection(
    b=400, 
    d=310, 
    d1=40, 
    d2=40, 
    concrete_material=cls, 
    As=Bars(n_bars=10, diameter=16, steel_material=steel), 
    As1=Bars(n_bars=5, diameter=16, steel_material=steel), 
    internal_forces=InternalForces(M=142.4*10**6, N=-4.4*10**3), 
    name="sec1"
    )
print(section_1)


Section name: sec1
   CLS = C25/30
     B = 400 mm
     H = 350 mm
     d = 310 mm
    d1 = 40 mm
    d2 = 40 mm
    As = 10Ø16 =  2010 mm2 | B450C
   As1 =  5Ø16 =  1005 mm2 | B450C
   Med = 142400000.0 Nmm
   Ned = -4400.0 N
   Ved = 0.0 N



### SLS:

In [4]:
results_dict, logs = sls(section=section_1)
print(results_dict)
print(logs)

{'x': 136.56295282180042, 'sigma_c': 14.038687258610103, 'sigma_s': 267.44022599987994, 'sigma_s1': 148.9002398625023, '<sigma_cR': True, '<sigma_sR': True, '<sigma_sR1': True}
Tenso-flessione


ULS:

In [5]:
results_dict, logs = computeVero(section=section_1)
print(results_dict)
#print(logs)

{'campo': '3A', 'es': 0.00924654547047127, 'es1': 0.00185528445542306, 'ec': 0.0035, 'xi': 0.274584200723884, 'psi': 0.8095238095238095, 'lamb': 0.41596638655462187, 'Nrd': -4399.99999999977, 'Mrd': 212923905.928571}


# Multiple sections ULS and SLS:

In [6]:
section_2 = ReinforcedConcreteSection(
    b=400, 
    d=310, 
    d1=40, 
    d2=40, 
    concrete_material=cls, 
    As=Bars(n_bars=5, diameter=16, steel_material=steel), 
    As1=Bars(n_bars=5, diameter=12, steel_material=steel), 
    internal_forces=InternalForces(M=26407.2*10**6, N=22*10**3), 
    name="sec2"
    )
section_3 = ReinforcedConcreteSection(
    b=500, 
    d=710, 
    d1=40, 
    d2=40, 
    concrete_material=cls, 
    As=Bars(n_bars=7, diameter=16, steel_material=steel), 
    As1=Bars(n_bars=5, diameter=16, steel_material=steel), 
    internal_forces=InternalForces(M=207.2*10**6, N=22*10**3), 
    name="sec3"
    )
list_of_sections = [section_1, section_2, section_3] 

In [7]:
input_dict = [section.to_dict() for section in list_of_sections]
row_names = [section.name for section in list_of_sections]
tab_input = pd.DataFrame(input_dict, index=row_names)


tab_input.drop(["name", "CLS", "Steel","Steel1", "Ved"], axis=1, inplace=True) # Remove those columns
tab_input["Med"] = tab_input["Med"].div(10**6) # Nmm -> kNm
tab_input["Ned"] = tab_input["Med"].div(10**3) # N -> kN
tab_input

,B,H,d,d1,d2,As_str,As_area,As1_str,As1_area,Med,Ned
sec1,400,350,310,40,40,10Ø16,2009.60,5Ø16,1004.8,142.4,0.1424
sec2,400,350,310,40,40,5Ø16,1004.80,5Ø12,565.2,26407.2,26.4072
sec3,500,750,710,40,40,7Ø16,1406.72,5Ø16,1004.8,207.2,0.2072


## ULS

In [8]:
list_of_dicts_results_ULS = []
for section in list_of_sections:
    results_dict, logs = computeVero(section=section)
    #print(logs)
    list_of_dicts_results_ULS.append(results_dict)
list_of_dicts_results_ULS

[{'campo': '3A',
  'es': 0.00924654547047127,
  'es1': 0.00185528445542306,
  'ec': 0.0035,
  'xi': 0.274584200723884,
  'psi': 0.8095238095238095,
  'lamb': 0.41596638655462187,
  'Nrd': -4399.99999999977,
  'Mrd': 212923905.928571},
 {'campo': '2A',
  'xi': 0.2148427370220405,
  'es1': 0.0010929081727151162,
  'ec': 0.002736301976080318,
  'es': 0.01,
  'psi': 0.756362173292858,
  'lamb': 0.39780111548266606,
  'Nrd': 21999.99999999936,
  'Mrd': 115954755.13631497},
 {'campo': '2A',
  'xi': 0.13395748978298877,
  'es1': 0.0008962546375988511,
  'ec': 0.001546777302530126,
  'es': 0.01,
  'psi': 0.5740119826298653,
  'lamb': 0.3622782384730227,
  'Nrd': 25379.08537669212,
  'Mrd': 387953153.84976166}]

In [9]:
uls_tab_check = pd.DataFrame(list_of_dicts_results_ULS, index=row_names)

uls_tab_tot = tab_input.join(uls_tab_check) 
uls_tab_tot.drop(["Nrd"], axis=1, inplace=True)
uls_tab_tot["Mrd"] = uls_tab_tot["Mrd"].div(10**6) # Nmm -> kNm
uls_tab_tot

,B,H,d,d1,d2,As_str,As_area,As1_str,As1_area,Med,Ned,campo,es,es1,ec,xi,psi,lamb,Mrd
sec1,400,350,310,40,40,10Ø16,2009.60,5Ø16,1004.8,142.4,0.1424,3A,0.00924654547047127,0.00185528445542306,0.003500,0.274584200723884,0.809524,0.415966,212.923905928571
sec2,400,350,310,40,40,5Ø16,1004.80,5Ø12,565.2,26407.2,26.4072,2A,0.01,0.001093,0.002736,0.214843,0.756362,0.397801,115.954755
sec3,500,750,710,40,40,7Ø16,1406.72,5Ø16,1004.8,207.2,0.2072,2A,0.01,0.000896,0.001547,0.133957,0.574012,0.362278,387.953154


## SLS

In [10]:
list_of_dicts_results_SLS = []
for section in list_of_sections:
    results_dict, logs = sls(section=section)
    print(logs)
    list_of_dicts_results_SLS.append(results_dict)
list_of_dicts_results_SLS

Tenso-flessione
Presso-flessione
Presso-flessione


[{'x': 136.56295282180042,
  'sigma_c': 14.038687258610103,
  'sigma_s': 267.44022599987994,
  'sigma_s1': 148.9002398625023,
  '<sigma_cR': True,
  '<sigma_sR': True,
  '<sigma_sR1': True},
 {'x': 110.02405095513399,
  'sigma_c': 3534.8694716920504,
  'sigma_s': 96372.86637077192,
  'sigma_s1': 33746.15066297396,
  '<sigma_cR': False,
  '<sigma_sR': False,
  '<sigma_sR1': False},
 {'x': 193.52300660308478,
  'sigma_c': 5.468088890542552,
  'sigma_s': 218.8997183890492,
  'sigma_s1': 65.06803466815589,
  '<sigma_cR': True,
  '<sigma_sR': True,
  '<sigma_sR1': True}]

In [11]:
sls_tab_check = pd.DataFrame(list_of_dicts_results_SLS, index=row_names)
sls_tab_tot = tab_input.join(sls_tab_check) #l'altro metodo è merge
sls_tab_tot

,B,H,d,d1,d2,As_str,As_area,As1_str,As1_area,Med,Ned,x,sigma_c,sigma_s,sigma_s1,<sigma_cR,<sigma_sR,<sigma_sR1
sec1,400,350,310,40,40,10Ø16,2009.60,5Ø16,1004.8,142.4,0.1424,136.562953,14.038687,267.440226,148.900240,True,True,True
sec2,400,350,310,40,40,5Ø16,1004.80,5Ø12,565.2,26407.2,26.4072,110.024051,3534.869472,96372.866371,33746.150663,False,False,False
sec3,500,750,710,40,40,7Ø16,1406.72,5Ø16,1004.8,207.2,0.2072,193.523007,5.468089,218.899718,65.068035,True,True,True


In [12]:
#STOP
s = uls_tab_tot.style
s.clear()
s.caption = "ULS"
s.format('{:.0f}', subset=["As_area","As1_area"])
s.format("{:.3f}", subset=["Med","Ned", "Mrd"])
s.format("{:.4f}", subset=["es","es1","ec", "xi","psi", "lamb"])

#with open(EXPORT_FOLDER+"prova.tex", 'w') as file:
 #   file.write(s.to_latex(position='H', hrules=True, siunitx=True))
s

,B,H,d,d1,d2,As_str,As_area,As1_str,As1_area,Med,Ned,campo,es,es1,ec,xi,psi,lamb,Mrd
sec1,400,350,310,40,40,10Ø16,2010,5Ø16,1005,142.400,0.142,3A,0.0092,0.0019,0.0035,0.2746,0.8095,0.4160,212.924
sec2,400,350,310,40,40,5Ø16,1005,5Ø12,565,26407.200,26.407,2A,0.0100,0.0011,0.0027,0.2148,0.7564,0.3978,115.955
sec3,500,750,710,40,40,7Ø16,1407,5Ø16,1005,207.200,0.207,2A,0.0100,0.0009,0.0015,0.1340,0.5740,0.3623,387.953


In [13]:
#STOP
s = sls_tab_tot.style
s.clear()
s.caption = "SLS"
s.format('{:.0f}', subset=["As_area","As1_area"])
s.format('{:.2f}', subset=["x",'sigma_c','sigma_s','sigma_s1'])
s.format("{:.3f}", subset=["Med","Ned"])

#with open(EXPORT_FOLDER+"prova.tex", 'w') as file:
 #   file.write(s.to_latex(position='H', hrules=True, siunitx=True))
s


,B,H,d,d1,d2,As_str,As_area,As1_str,As1_area,Med,Ned,x,sigma_c,sigma_s,sigma_s1,<sigma_cR,<sigma_sR,<sigma_sR1
sec1,400,350,310,40,40,10Ø16,2010,5Ø16,1005,142.400,0.142,136.56,14.04,267.44,148.90,True,True,True
sec2,400,350,310,40,40,5Ø16,1005,5Ø12,565,26407.200,26.407,110.02,3534.87,96372.87,33746.15,False,False,False
sec3,500,750,710,40,40,7Ø16,1407,5Ø16,1005,207.200,0.207,193.52,5.47,218.90,65.07,True,True,True


In [14]:
s.to_latex()

'\\begin{table}\n\\caption{SLS}\n\\begin{tabular}{lrrrrrlrlrrrrrrrlll}\n & B & H & d & d1 & d2 & As_str & As_area & As1_str & As1_area & Med & Ned & x & sigma_c & sigma_s & sigma_s1 & <sigma_cR & <sigma_sR & <sigma_sR1 \\\\\nsec1 & 400 & 350 & 310 & 40 & 40 & 10Ø16 & 2010 & 5Ø16 & 1005 & 142.400 & 0.142 & 136.56 & 14.04 & 267.44 & 148.90 & True & True & True \\\\\nsec2 & 400 & 350 & 310 & 40 & 40 & 5Ø16 & 1005 & 5Ø12 & 565 & 26407.200 & 26.407 & 110.02 & 3534.87 & 96372.87 & 33746.15 & False & False & False \\\\\nsec3 & 500 & 750 & 710 & 40 & 40 & 7Ø16 & 1407 & 5Ø16 & 1005 & 207.200 & 0.207 & 193.52 & 5.47 & 218.90 & 65.07 & True & True & True \\\\\n\\end{tabular}\n\\end{table}\n'